In [13]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import random
#Load labels, prediction, and probabilities
def load_prediction_data(prediction_folder, label_folder):
    all_data = []
    label_files = [f for f in sorted(listdir(label_folder)) if isfile(join(label_folder, f))]
    for idx, input_file in enumerate(label_files):
        data = pd.read_csv(label_folder+input_file, delimiter='|', header=0)
        all_data.append(data)
        if idx % 500 == 0:
            print("Finished ", idx)
    

    cohort_labels = []
    for data in all_data:
        label = data['SepsisLabel'].values
        cohort_labels.append(label)
    #load predictions/probabilities
    all_output_data = []
    output_files = [f for f in sorted(listdir(prediction_folder)) if isfile(join(prediction_folder, f))]
    for idx, output_file in enumerate(output_files):
        data = pd.read_csv(prediction_folder+output_file, delimiter='|', header=0)
        all_output_data.append(data)
        if idx % 500 == 0:
            print("Finished ", idx)
    cohort_probabilities = []
    cohort_predictions = []
    for data in all_output_data:
        probability = data['PredictedProbability']
        prediction = data['PredictedLabel']
        cohort_probabilities.append(probability)
        cohort_predictions.append(prediction)
    return cohort_labels, cohort_probabilities, cohort_predictions

In [14]:
#prediction_folder= '/home/luan/Windows_Data2/sepsis_detection/sample_output/'
#label_folder = '/home/luan/Windows_Data2/sepsis_detection/sample_test/'

prediction_folder= '/home/manhnh/Downloads/predict/'
label_folder = '/home/manhnh/Downloads/label/'

cohort_labels, cohort_probabilities, cohort_predictions = load_prediction_data(prediction_folder, label_folder)

Finished  0
Finished  500
Finished  0
Finished  500


In [15]:
#Compute utility
def compute_auc(labels, predictions):
    # Check inputs for errors.
    if len(predictions) != len(labels):
        raise Exception('Numbers of predictions and labels must be the same.')

    n = len(labels)
    for i in range(n):
        if not labels[i] in (0, 1):
            raise Exception('Labels must satisfy label == 0 or label == 1.')

    for i in range(n):
        if not 0 <= predictions[i] <= 1:
            raise Exception('Predictions must satisfy 0 <= prediction <= 1.')

    # Find prediction thresholds.
    thresholds = np.unique(predictions)[::-1]
    if thresholds[0] != 1:
        thresholds = np.concatenate((np.array([1]), thresholds))

    if thresholds[-1] != 0:
        thresholds = np.concatenate((thresholds, np.array([0])))
    m = len(thresholds)

    # Populate contingency table across prediction thresholds.
    tp = np.zeros(m)
    fp = np.zeros(m)
    fn = np.zeros(m)
    tn = np.zeros(m)

    # Find indices that sort predicted probabilities from largest to smallest.
    idx = np.argsort(predictions)[::-1]

    i = 0
    for j in range(m):
        # Initialize contingency table for j-th prediction threshold.
        if j == 0:
            tp[j] = 0
            fp[j] = 0
            fn[j] = np.sum(labels == 1)
            tn[j] = np.sum(labels == 0)
        else:
            tp[j] = tp[j - 1]
            fp[j] = fp[j - 1]
            fn[j] = fn[j - 1]
            tn[j] = tn[j - 1]

        # Update contingency table for i-th largest prediction probability.
        while i < n and predictions[idx[i]] >= thresholds[j]:
            if labels[idx[i]]:
                tp[j] += 1
                fn[j] -= 1
            else:
                fp[j] += 1
                tn[j] -= 1
            i += 1

    # Summarize contingency table.
    tpr = np.zeros(m)
    tnr = np.zeros(m)
    ppv = np.zeros(m)
    npv = np.zeros(m)

    for j in range(m):
        if tp[j] + fn[j]:
            tpr[j] = tp[j] / (tp[j] + fn[j])
        else:
            tpr[j] = 1
        if fp[j] + tn[j]:
            tnr[j] = tn[j] / (fp[j] + tn[j])
        else:
            tnr[j] = 1
        if tp[j] + fp[j]:
            ppv[j] = tp[j] / (tp[j] + fp[j])
        else:
            ppv[j] = 1
        if fn[j] + tn[j]:
            npv[j] = tn[j] / (fn[j] + tn[j])
        else:
            npv[j] = 1

    # Compute AUROC as the area under a piecewise linear function of TPR /
    # sensitivity (x-axis) and TNR / specificity (y-axis) and AUPRC as the area
    # under a piecewise constant of TPR / recall (x-axis) and PPV / precision
    # (y-axis).
    auroc = 0
    auprc = 0
    for j in range(m-1):
        auroc += 0.5 * (tpr[j + 1] - tpr[j]) * (tnr[j + 1] + tnr[j])
        auprc += (tpr[j + 1] - tpr[j]) * ppv[j + 1]

    return auroc, auprc
def compute_accuracy_f_measure(labels, predictions):
    # Check inputs for errors.
    if len(predictions) != len(labels):
        raise Exception('Numbers of predictions and labels must be the same.')

    n = len(labels)
    for i in range(n):
        if not labels[i] in (0, 1):
            raise Exception('Labels must satisfy label == 0 or label == 1.')

    for i in range(n):
        if not predictions[i] in (0, 1):
            raise Exception(
                'Predictions must satisfy prediction == 0 or prediction == 1.')

    # Populate contingency table.
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i in range(n):
        if labels[i] and predictions[i]:
            tp += 1
        elif labels[i] and not predictions[i]:
            fp += 1
        elif not labels[i] and predictions[i]:
            fn += 1
        elif not labels[i] and not predictions[i]:
            tn += 1

    # Summarize contingency table.
    if tp + fp + fn + tn:
        accuracy = float(tp + tn) / float(tp + fp + fn + tn)
    else:
        accuracy = 1.0

    if 2 * tp + fp + fn:
        f_measure = float(2 * tp) / float(2 * tp + fp + fn)
    else:
        f_measure = 1.0

    return accuracy, f_measure

def compute_prediction_utility(labels, predictions, dt_early=-12, dt_optimal=-6, dt_late=3.0, max_u_tp=1, min_u_fn=-2, u_fp=-0.05, u_tn=0):
    # Check inputs for errors.
    if len(predictions) != len(labels):
        raise Exception('Numbers of predictions and labels must be the same.')

    n = len(labels)
    for i in range(n):
        if not labels[i] in (0, 1):
            raise Exception('Labels must satisfy label == 0 or label == 1.')

    for i in range(n):
        if not predictions[i] in (0, 1):
            raise Exception(
                'Predictions must satisfy prediction == 0 or prediction == 1.')

    if dt_early >= dt_optimal:
        raise Exception(
            'The earliest beneficial time for predictions must be before the optimal time.')

    if dt_optimal >= dt_late:
        raise Exception(
            'The optimal time for predictions must be before the latest beneficial time.')

    # Does the patient eventually have sepsis?
    if any(labels):
        is_septic = True
        t_sepsis = min(i for i, label in enumerate(
            labels) if label) - dt_optimal
    else:
        is_septic = False
        t_sepsis = float('inf')

    # Define slopes and intercept points for affine utility functions of the
    # form u = m * t + b.
    m_1 = float(max_u_tp) / float(dt_optimal - dt_early)
    b_1 = -m_1 * dt_early
    m_2 = float(-max_u_tp) / float(dt_late - dt_optimal)
    b_2 = -m_2 * dt_late
    m_3 = float(min_u_fn) / float(dt_late - dt_optimal)
    b_3 = -m_3 * dt_optimal

    # Compare predicted and true conditions.
    u = np.zeros(n)
    for t in range(n):
        if t <= t_sepsis + dt_late:
            # TP
            if is_septic and predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = max(m_1 * (t - t_sepsis) + b_1, u_fp)
                elif t <= t_sepsis + dt_late:
                    u[t] = m_2 * (t - t_sepsis) + b_2
            # FN
            elif is_septic and not predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = 0
                elif t <= t_sepsis + dt_late:
                    u[t] = m_3 * (t - t_sepsis) + b_3
            # FP
            elif not is_septic and predictions[t]:
                u[t] = u_fp
            # TN
            elif not is_septic and not predictions[t]:
                u[t] = u_tn

    # Find total utility for patient.
    return np.sum(u)


In [16]:
labels = np.concatenate(cohort_labels)
predictions = np.concatenate(cohort_predictions)
probabilities = np.concatenate(cohort_probabilities)

auroc, auprc = compute_auc(labels, probabilities)
accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)

print("AUCROC, AUCPRC")
print(auroc, auprc)
print("Accuaracy, F-measure")
print(accuracy, f_measure) 

num_files = len(cohort_labels)
dt_early = -12
dt_optimal = -6
dt_late = 3

max_u_tp = 1
min_u_fn = -2
u_fp = -0.05
u_tn = 0
# Compute utility.
observed_utilities = np.zeros(num_files)
best_utilities = np.zeros(num_files)
worst_utilities = np.zeros(num_files)
inaction_utilities = np.zeros(num_files)

for k in range(num_files):
    labels = cohort_labels[k]
    num_records = len(labels)
    observed_predictions = cohort_predictions[k]
    best_predictions = np.zeros(num_records)
    worst_predictions = np.zeros(num_records)
    inaction_predictions = np.zeros(num_records)

    if any(labels):
        t_sepsis = min(i for i, label in enumerate(
            labels) if label) - dt_optimal
        best_predictions[max(0, t_sepsis + dt_early)
                             : min(t_sepsis + dt_late, num_records)] = 1
    else:
        best_predictions[:] = 0
    worst_predictions = 1 - best_predictions

    observed_utilities[k] = compute_prediction_utility(
        labels, observed_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    best_utilities[k] = compute_prediction_utility(
        labels, best_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    worst_utilities[k] = compute_prediction_utility(
        labels, worst_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
    inaction_utilities[k] = compute_prediction_utility(
        labels, inaction_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)

unnormalized_observed_utility = np.sum(observed_utilities)
unnormalized_best_utility = np.sum(best_utilities)
unnormalized_worst_utility = np.sum(worst_utilities)
unnormalized_inaction_utility = np.sum(inaction_utilities)

if not (unnormalized_worst_utility <= unnormalized_best_utility and unnormalized_inaction_utility <= unnormalized_best_utility):
    raise Exception(
        'Optimal utility must be higher than inaction utility.')

normalized_observed_utility = (unnormalized_observed_utility - unnormalized_inaction_utility) / (
    unnormalized_best_utility - unnormalized_inaction_utility)

print("Utility, ", normalized_observed_utility)

AUCROC, AUCPRC
0.8706202294500328 0.15396754459579734
Accuaracy, F-measure
0.8388002782859646 0.17051180058339963
Utility,  0.5816521667760999
